### Setup

In [1]:
import json
import faiss
import numpy as np
import chess
import chess.svg
import tensorflow as tf

from chesspos.convert import bitboard_to_board
from chesspos.binary_index import board_to_bitboard
from chesspos.utils import files_from_directory
import chesspos.embedding_index as iemb

In [2]:
train_path = "/media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_train"
embedding_path = "/media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add"
encoder_path = "/home/pafrank/Documents/coding/chess-position-embedding/metric_learning/deep64/model_encoder.h5"
decoder_path ="/home/pafrank/Documents/coding/chess-position-embedding/metric_learning/deep64/model_decoder.h5"
save_path = "/media/pafrank/Backup/other/Chess/lichess/embeddings"
table_id = "test_embedding"
queries = [
    "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1",
    "8/1R6/4p1k1/1p6/p1b2K2/P1Br4/1P6/8 b - - 8 49"
]
num_results = 10
embedding_dimension = 64

### Create and populate index

In [3]:
# create index
index = faiss.index_factory(embedding_dimension, "PCA16,SQ4")

In [4]:
%%time
# train faiss index
train_file_list = files_from_directory(embedding_path, file_type="h5")
index = iemb.index_train_embeddings(train_file_list, table_id, index, train_frac=1e-3)

File /media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add/lichess_db_standard_rated_2013-01-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1', 'test_embedding_0', 'test_embedding_1']>
File /media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add/lichess_db_standard_rated_2013-02-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1', 'test_embedding_0', 'test_embedding_1']>
File /media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add/lichess_db_standard_rated_2013-03-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1', 'test_embedding_0', 'test_embedding_1']>
Training on 27370 positions
faiss index trained? True
CPU times: user 5.02 s, sys: 2.43 s, total: 7.45 s
Wall time: 7.46 s


In [5]:
%%time
# populate faiss index
file_list = files_from_directory(embedding_path, file_type="h5")
index, table_dict = iemb.index_load_file_array(file_list, table_id, index, chunks=int(1e5))

File /media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add/lichess_db_standard_rated_2013-01-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1', 'test_embedding_0', 'test_embedding_1']>
File /media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add/lichess_db_standard_rated_2013-02-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1', 'test_embedding_0', 'test_embedding_1']>
File /media/pafrank/Backup/other/Chess/lichess/embeddings/bb_d64_add/lichess_db_standard_rated_2013-03-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1', 'test_embedding_0', 'test_embedding_1']>
CPU times: user 2min 46s, sys: 14.8 s, total: 3min 1s
Wall time: 2min 44s


In [6]:
%%time
# save index
faiss.write_index(index, f"{save_path}/bb_d64_pca16_sq4.faiss")
json.dump( table_dict, open( f"{save_path}/bb_d64_pca16_sq4.json", 'w' ) )

CPU times: user 439 ms, sys: 236 ms, total: 675 ms
Wall time: 1.83 s


### Search index

In [7]:
queries = np.array([
    "r1bqk1nr/pp1pbppp/2n1p3/8/3N4/6P1/PPP1PPBP/RNBQK2R w KQkq - 3 6",
    "8/1R6/4p1k1/1p6/p1b2K2/P1Br4/1P6/8 b - - 8 49",
    "8/8/5p2/R3pkp1/5n2/5K2/8/8 w - - 0 42"
])
num_results = 3
table_dict = json.load( open( f"{save_path}/bb_d64_pca16_sq4.json" ) ) # so that keys are strings

In [11]:
%%time
# search index
D, I, E = iemb.index_query_positions(queries, index, encoder_path,
                                     input_format='fen', num_results=num_results)

CPU times: user 10 s, sys: 53.9 ms, total: 10.1 s
Wall time: 5.29 s


In [12]:
%%time
# retrieve the belonging bitboards
file, table, offset = iemb.location_from_index(I, table_dict)
bb_table = iemb.manipulate_prefix(table, "position")
bitboards = iemb.retrieve_elements_from_file(file, bb_table, offset)
print(bitboards.shape, bitboards.dtype)

[ 6717385  8155187 14883482 16489287 23332694 27346630]
(3, 3, 773) bool
CPU times: user 468 ms, sys: 5.11 ms, total: 473 ms
Wall time: 1.4 s


In [13]:
# retrieve belonging embeddings
embeddings = iemb.retrieve_elements_from_file(file, table, offset)
e_shape = embeddings.shape
print(f"embedding shape {e_shape}")
embeddings = embeddings.reshape((-1,e_shape[-1]))

# reconstruct with decoder
decoder = tf.keras.models.load_model(decoder_path)
decoded_bitboards = decoder(embeddings)
decoded_bitboards = decoded_bitboards.numpy()
decoded_bitboards = decoded_bitboards.reshape((*e_shape[:-1],-1))
print(f"reconstructed bitboard shape {decoded_bitboards.shape}")

embedding shape (3, 3, 64)
reconstructed bitboard shape (3, 3, 773)


In [14]:
# convert bitboards to fen
def fen_converter(bb):
    board = bitboard_to_board(bb) 
    return board.fen()
fc = np.vectorize(fen_converter, signature=f'(773)->()')

bitboards_fen = fc(bitboards)
decoded_bitboards_fen = fc(decoded_bitboards)

bb_shape = bitboards.shape

In [15]:
from IPython.display import HTML
html = ""
for i in range(bb_shape[0]):
    for j in range(bb_shape[1]):
        html += f"<h4>Query {i} | Retrieved bitboard {j}: euclidean distance {D[i][j]} to query | Reconstructed bitboard {j}</h4>"
        html += chess.svg.board(chess.Board(queries[i]), size=300)
        html += chess.svg.board(chess.Board(bitboards_fen[i][j]), size=300)
        html += chess.svg.board(chess.Board(decoded_bitboards_fen[i][j]), size=300)
    html += "<hr>"
HTML(html)